In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

from mixcoatl.database import db_session, query_results
from mixcoatl.utils import CrosstalkResults

# Displaying Crosstalk Results

MixCOATL uses a SQLite database in order to organize and store the results of crosstalk analyses. For more information on the database structure and a brief introduction to interfacing with the SQLite database using MixCOATL tools or Python built-in methods, see the notebook `Example_DatabaseIntro.ipynb`.

Here, we would like to be able to view some crosstalk results taken for one half of a CCD (Amp1-Amp8).  There are a number of different combinations of aggressor and victim amplifiers, so an IPython interactive widget is used to faciliate viewing of the results.  First, the database is queried for all results of the CCD (`sensor_name = 'Davis'`) that were measured using the least squares minimization method (`method='MODEL_LSQ'`). NOTE: the `for` loops are made only over the half of the CCD that has measurements.

In [2]:
database = './data/example_crosstalk_database.db'

## Query database for results of each aggressor/victim pairing
results = {}
for agg in range(1, 9):
    for vic in range(1, 9):
        
        ## Sort results by aggressor signal and add to dictionary
        with db_session(database) as f:

            r = query_results(f, 'Davis', agg, vic, methods='MODEL_LSQ')
            r.sort(key=lambda x: x.aggressor_signal) 
            
            coefficients = np.asarray([result.coefficient for result in r])
            error = np.asarray([result.error for result in r])
            signals = np.asarray([result.aggressor_signal for result in r])
            results[(agg, vic)] = (signals, coefficients, error)

Next a `CrosstalkResults` object is initialized with the results dictionary (`results`) and an aggressor/victim amplifier pair (`agg0`, `vic0`). This creates an interactive plot that has a number of widget features to control the display of crosstalk results:

1. A slider to select the Aggressor amplifier number.
2. A slider to select the Victim amplifier number.
3. A checkbox to toggle between absolute crosstalk coefficient and a normalized crosstalk coefficient (NOTE: normalization is done using the highest signal crosstalk measurement).

Changes made using any of these three widgets will automatically regenerate a graph of the new results, rescale the y-axis, and update labels. 

The results presented include error bars, although in most aggressor/victim amplifier pairings they are too small to see.

In [3]:
## Make results display widget
agg0, vic0 = 1, 2

CrosstalkResults(results, agg0, vic0)

CrosstalkResults(children=(Output(), TwoByTwoLayout(children=(IntSlider(value=1, continuous_update=False, desc…

To close the widget (e.g. to remove the plots from memory), simply close the plot.

In [4]:
plt.close()